In [ ]:
from _util import *
import _IS as IS
import _RL.FQI as FQI
import _RL.my_gym as my_gym
import _Ohio_Simulator as Ohio
import _analyze as analyze
import _RL.DQN as DQN
import _cartpole as cartpole
from coinDice import run_neural_coin_dice as run_coin

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
from arch.bootstrap import IIDBootstrap

reload(run_coin)
reload(my_gym)
reload(IS)

1. we can either store the data generated from TR, or re-generate it 

In [ ]:
################################################################################################
############################## generate data ##############################################
################################################################################################
unique_setting = "ohio_IS"
for N in [10, 15, 20, 25, 30]:
    tau = 1
    ###################################################################################################################
    n_gpu = 8
    rep = 200

    tp_path = "target_policies/1003_gamma95_sd10/1003_gamma95_sd10_50"
    ########################################################################
    sd_G = 10
    T = 200
    gamma = .95
    ################################################################################################################################################ 
    ###################################################### Default Settings ######################################################
    ################################################################################################################################################
    setting = EST()[7:9] + EST()[10:13]+ "_"  + unique_setting + "_" + "N" + str(N) + "_" + "T" + str(T) + "_" + "sd" + str(sd_G)  + "_"  + "tau" + "_" + str(tau)
    printR("setting: {}".format(setting))
    ################################################################################################################################################

    ####################################
    FQI_paras = {"hiddens" : [64, 64], "gamma" : gamma}
    pi1 = FQI.FQI(gpu_number = 0, **FQI_paras)
    pi1.model.load_weights(tp_path)
    pi_behva = my_gym.softmax_policy(pi1, tau)
    ####################################
    ohio = Ohio.OhioSimulator(sd_G = sd_G, T = T, N = N, behav = pi_behva)
    V_true = ohio.eval_policy(pi = pi1, gamma = gamma, N = 10000)
    init_S = ohio.simu_init_S(seed = 0, N = 1000)
    ohio.eval_policy(pi = pi_behva, gamma = gamma, N = 1000)
    ######################################################################################## 
    trajs_train_resp = Ohio.OhioSimulator(sd_G = sd_G, T = T, N = N * 500, behav = pi_behva).simu_one_seed(10000)
    trajs_train_resp = [trajs_train_resp[(i * N):((i + 1) * N)] for i in range(rep)]

    ################################################################################################
    ############################## calculate step-wise IS ##############################################
    ################################################################################################
    V_seeds = IS.cal_step_IS(trajs_train_resp, gamma, pi_behva, pi1)
    ################################################################################################
    ############################## calculate CIs ##############################################
    ################################################################################################
    ######################################  
    ## Bernstein_CI: perfect but too wide
    CI_seeds = []
    cnt_09, cnt_095, width09, width095 = [], [], [], []
    for V_trajs, V_min, V_max in V_seeds:
        CIs_this = IS.cal_Bernstein_CI(Ys = V_trajs, V_min = V_min, V_max = V_max)
        CI_seeds.append(CIs_this) # [90%, 95%]
        CI09, CI095 = CIs_this
        cnt_09.append((CI09[0] < V_true and CI09[1] > V_true))
        width09.append(CI09[1] - CI09[0])
        cnt_095.append((CI095[0] < V_true and CI095[1] > V_true))
        width095.append(CI095[1] - CI095[0])
    res_IS_Bernstein = {"all_CI" : CI_seeds, "width" : [mean(width09), mean(width095)], "freq" : [mean(cnt_09), mean(cnt_095)]}

    dump(res_IS_Bernstein, "res/" + unique_setting + "_Bernstein" + "_N" + str(N) + "_tau" + str(tau))
    ######################################
    ## Bootstrap CIs
    CI_seeds = []
    cnt095, cnt09, width09, width095 = [], [], [], []
    for seed in range(200):
        V_trajs, _, _ = V_seeds[seed]
        bs = IIDBootstrap(arr(V_trajs))
        ci095 = bs.conf_int(lambda x: x.mean(), 1000, method='bca', size = 0.95)
        ci09 = bs.conf_int(lambda x: x.mean(), 1000, method='bca', size = 0.9)
        CI_seeds.append([ci09, ci095])
        cnt095.append((ci095[0][0] < V_true and ci095[1][0] > V_true))
        cnt09.append((ci09[0][0] < V_true and ci09[1][0] > V_true))
        width09.append(ci09[1] - ci09[0])
        width095.append(ci095[1] - ci095[0])
    res_IS_Bootstrap = {"all_CI" : CI_seeds, "width" : [mean(width09), mean(width095)], "freq" : [mean(cnt09), mean(cnt095)]}

    ################################################################################################
    ############################## save results ##############################################
    ################################################################################################
    dump(res_IS_Bootstrap, "res/" + unique_setting + "_Bootstrap" + "_N" + str(N) + "_tau" + str(tau))

In [ ]:
################################################################################################
############################## generate data ##############################################
################################################################################################
unique_setting = "ohio_IS"
N = 20
for tau in [0.8, 0.9, 1.1, 1.2]:
    ###################################################################################################################
    n_gpu = 8
    rep = 200

    tp_path = "target_policies/1003_gamma95_sd10/1003_gamma95_sd10_50"
    ########################################################################
    sd_G = 10
    T = 200
    gamma = .95
    ################################################################################################################################################ 
    ###################################################### Default Settings ######################################################
    ################################################################################################################################################
    setting = EST()[7:9] + EST()[10:13]+ "_"  + unique_setting + "_" + "N" + str(N) + "_" + "T" + str(T) + "_" + "sd" + str(sd_G)  + "_"  + "tau" + "_" + str(tau)
    printR("setting: {}".format(setting))
    ################################################################################################################################################

    ####################################
    FQI_paras = {"hiddens" : [64, 64], "gamma" : gamma}
    pi1 = FQI.FQI(gpu_number = 0, **FQI_paras)
    pi1.model.load_weights(tp_path)
    pi_behva = my_gym.softmax_policy(pi1, tau)
    ####################################
    ohio = Ohio.OhioSimulator(sd_G = sd_G, T = T, N = N, behav = pi_behva)
    V_true = ohio.eval_policy(pi = pi1, gamma = gamma, N = 10000)
    init_S = ohio.simu_init_S(seed = 0, N = 1000)
    ohio.eval_policy(pi = pi_behva, gamma = gamma, N = 1000)
    ######################################################################################## 
    trajs_train_resp = Ohio.OhioSimulator(sd_G = sd_G, T = T, N = N * 500, behav = pi_behva).simu_one_seed(10000)
    trajs_train_resp = [trajs_train_resp[(i * N):((i + 1) * N)] for i in range(rep)]

    ################################################################################################
    ############################## calculate step-wise IS ##############################################
    ################################################################################################
    V_seeds = IS.cal_step_IS(trajs_train_resp, gamma, pi_behva, pi1)
    ################################################################################################
    ############################## calculate CIs ##############################################
    ################################################################################################
    ######################################  
    ## Bernstein_CI: perfect but too wide
    CI_seeds = []
    cnt_09, cnt_095, width09, width095 = [], [], [], []
    for V_trajs, V_min, V_max in V_seeds:
        CIs_this = IS.cal_Bernstein_CI(Ys = V_trajs, V_min = V_min, V_max = V_max)
        CI_seeds.append(CIs_this) # [90%, 95%]
        CI09, CI095 = CIs_this
        cnt_09.append((CI09[0] < V_true and CI09[1] > V_true))
        width09.append(CI09[1] - CI09[0])
        cnt_095.append((CI095[0] < V_true and CI095[1] > V_true))
        width095.append(CI095[1] - CI095[0])
    res_IS_Bernstein = {"all_CI" : CI_seeds, "width" : [mean(width09), mean(width095)], "freq" : [mean(cnt_09), mean(cnt_095)]}

    dump(res_IS_Bernstein, "res/" + unique_setting + "_Bernstein" + "_N" + str(N) + "_tau" + str(tau))
    ######################################
    ## Bootstrap CIs
    CI_seeds = []
    cnt095, cnt09, width09, width095 = [], [], [], []
    for seed in range(200):
        V_trajs, _, _ = V_seeds[seed]
        bs = IIDBootstrap(arr(V_trajs))
        ci095 = bs.conf_int(lambda x: x.mean(), 1000, method='bca', size = 0.95)
        ci09 = bs.conf_int(lambda x: x.mean(), 1000, method='bca', size = 0.9)
        CI_seeds.append([ci09, ci095])
        cnt095.append((ci095[0][0] < V_true and ci095[1][0] > V_true))
        cnt09.append((ci09[0][0] < V_true and ci09[1][0] > V_true))
        width09.append(ci09[1] - ci09[0])
        width095.append(ci095[1] - ci095[0])
    res_IS_Bootstrap = {"all_CI" : CI_seeds, "width" : [mean(width09), mean(width095)], "freq" : [mean(cnt09), mean(cnt095)]}

    ################################################################################################
    ############################## save results ##############################################
    ################################################################################################
    dump(res_IS_Bootstrap, "res/" + unique_setting + "_Bootstrap" + "_N" + str(N) + "_tau" + str(tau))